# World Bank Data Pipeline with Dagster + dbt + Snowflake

This notebook demonstrates a complete ETL pipeline for World Bank Open Data:
1. **Extract**: Fetch data from World Bank API
2. **Transform**: Clean and transform data using dbt
3. **Load**: Load data into Snowflake
4. **Orchestrate**: Use Dagster to orchestrate the pipeline

## 📋 Prerequisites
- Snowflake account with credentials
- Python packages: `requests`, `pandas`, `dagster`, `dbt-core`, `dbt-snowflake`, `snowflake-connector-python`

## 1. Import Required Libraries

In [17]:
import os
import json
import time
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Any

import requests
import pandas as pd
import snowflake.connector
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✓ Libraries imported successfully")
print(f"Current working directory: {Path.cwd()}")

✓ Libraries imported successfully
Current working directory: c:\Users\ahmad\Data-Exploration-Preparation-and-Visualization-final-project\notebooks


## 2. Secure Password Input

Enter your Snowflake password securely (it won't be displayed or stored).

In [18]:
import getpass

# Securely prompt for Snowflake password
print("🔒 Please enter your Snowflake password:")
SNOWFLAKE_PASSWORD = getpass.getpass("Password: ")
print("✅ Password captured securely (not displayed or stored)")

🔒 Please enter your Snowflake password:
✅ Password captured securely (not displayed or stored)
✅ Password captured securely (not displayed or stored)


## 2. Configure World Bank API & Constants

In [19]:
# World Bank API Configuration
BASE_URL = "https://api.worldbank.org/v2"
DATA_DIR = Path("../data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Popular indicators to fetch
INDICATORS = {
    "SP.POP.TOTL": "Population, total",
    "NY.GDP.MKTP.CD": "GDP (current US$)",
    "NY.GDP.PCAP.CD": "GDP per capita (current US$)",
    "SP.DYN.LE00.IN": "Life expectancy at birth, total (years)",
    "SE.ADT.LITR.ZS": "Literacy rate, adult total (%)",
    "SH.DYN.MORT": "Mortality rate, under-5 (per 1,000 live births)",
    "EN.ATM.CO2E.PC": "CO2 emissions (metric tons per capita)"
}

# Countries to analyze (Middle East & Africa)
COUNTRIES = ["EG", "SA", "AE", "JO", "NG", "ZA", "KE"]
YEAR_RANGE = "2010:2023"

print("✓ Configuration set")
print(f"Data directory: {DATA_DIR.absolute()}")
print(f"Countries: {', '.join(COUNTRIES)}")
print(f"Year range: {YEAR_RANGE}")

✓ Configuration set
Data directory: c:\Users\ahmad\Data-Exploration-Preparation-and-Visualization-final-project\notebooks\..\data
Countries: EG, SA, AE, JO, NG, ZA, KE
Year range: 2010:2023


## 3. Define Data Fetching Functions

In [20]:
def fetch_indicator_data(indicator: str, countries: List[str], date_range: str, per_page: int = 1000) -> List[Dict[str, Any]]:
    """Fetch indicator data from World Bank API."""
    all_data = []
    country_str = ";".join(countries)
    url = f"{BASE_URL}/country/{country_str}/indicator/{indicator}"
    
    params = {"format": "json", "date": date_range, "per_page": per_page, "page": 1}
    
    page = 1
    while True:
        params["page"] = page
        try:
            response = requests.get(url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()
            
            if len(data) < 2 or data[1] is None:
                break
            
            records = data[1]
            if not records:
                break
            
            all_data.extend(records)
            
            metadata = data[0]
            total_pages = metadata.get("pages", 1)
            if page >= total_pages:
                break
            
            page += 1
            time.sleep(0.5)  # Rate limiting
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            break
    
    return all_data


def transform_worldbank_data(raw_data: List[Dict[str, Any]]) -> pd.DataFrame:
    """Transform World Bank API response to clean DataFrame."""
    transformed = []
    fetched_at = datetime.now().isoformat()
    
    for record in raw_data:
        if record.get("value") is None:
            continue
        
        transformed_record = {
            "indicator_id": record.get("indicator", {}).get("id"),
            "indicator_name": record.get("indicator", {}).get("value"),
            "country_id": record.get("country", {}).get("id"),
            "country_name": record.get("country", {}).get("value"),
            "year": int(record.get("date", 0)),
            "value": float(record.get("value", 0)),
            "unit": record.get("unit", ""),
            "obs_status": record.get("obs_status", ""),
            "fetched_at": fetched_at
        }
        transformed.append(transformed_record)
    
    return pd.DataFrame(transformed)

print("✓ Functions defined")

✓ Functions defined


## 4. Fetch Data from World Bank API

Let's fetch multiple indicators for our countries of interest.

In [21]:
# Fetch data for all indicators
all_dataframes = []

for indicator_code, indicator_name in INDICATORS.items():
    print(f"\nFetching: {indicator_name} ({indicator_code})")
    
    raw_data = fetch_indicator_data(indicator_code, COUNTRIES, YEAR_RANGE)
    
    if raw_data:
        df = transform_worldbank_data(raw_data)
        all_dataframes.append(df)
        print(f"  ✓ Fetched {len(df)} records")
    else:
        print(f"  ✗ No data available")

# Combine all data into one DataFrame
if all_dataframes:
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    print(f"\n{'='*60}")
    print(f"✓ Total records fetched: {len(combined_df)}")
    print(f"  Countries: {combined_df['country_id'].nunique()}")
    print(f"  Indicators: {combined_df['indicator_id'].nunique()}")
    print(f"  Year range: {combined_df['year'].min()} - {combined_df['year'].max()}")
    print(f"{'='*60}")
else:
    print("✗ No data fetched")


Fetching: Population, total (SP.POP.TOTL)
  ✓ Fetched 98 records

Fetching: GDP (current US$) (NY.GDP.MKTP.CD)
  ✓ Fetched 98 records

Fetching: GDP (current US$) (NY.GDP.MKTP.CD)
  ✓ Fetched 98 records

Fetching: GDP per capita (current US$) (NY.GDP.PCAP.CD)
  ✓ Fetched 98 records

Fetching: GDP per capita (current US$) (NY.GDP.PCAP.CD)
  ✓ Fetched 98 records

Fetching: Life expectancy at birth, total (years) (SP.DYN.LE00.IN)
  ✓ Fetched 98 records

Fetching: Life expectancy at birth, total (years) (SP.DYN.LE00.IN)
  ✓ Fetched 98 records

Fetching: Literacy rate, adult total (%) (SE.ADT.LITR.ZS)
  ✓ Fetched 98 records

Fetching: Literacy rate, adult total (%) (SE.ADT.LITR.ZS)
  ✓ Fetched 28 records

Fetching: Mortality rate, under-5 (per 1,000 live births) (SH.DYN.MORT)
  ✓ Fetched 28 records

Fetching: Mortality rate, under-5 (per 1,000 live births) (SH.DYN.MORT)
  ✓ Fetched 98 records

Fetching: CO2 emissions (metric tons per capita) (EN.ATM.CO2E.PC)
  ✓ Fetched 98 records

Fetchin

## 5. Explore and Preview Data

In [22]:
# Display basic information
print("Data Shape:", combined_df.shape)
print("\nColumn Types:")
print(combined_df.dtypes)
print("\n" + "="*60)
print("First 10 rows:")
combined_df.head(10)

Data Shape: (518, 9)

Column Types:
indicator_id       object
indicator_name     object
country_id         object
country_name       object
year                int64
value             float64
unit               object
obs_status         object
fetched_at         object
dtype: object

First 10 rows:


,indicator_id,indicator_name,country_id,country_name,year,value,unit,obs_status,fetched_at
0,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2023,10483751.0,,,2025-10-30T11:31:41.660849
1,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2022,10074977.0,,,2025-10-30T11:31:41.660849
2,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2021,9575152.0,,,2025-10-30T11:31:41.660849
3,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2020,9401038.0,,,2025-10-30T11:31:41.660849
4,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2019,9445785.0,,,2025-10-30T11:31:41.660849
5,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2018,9346701.0,,,2025-10-30T11:31:41.660849
6,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2017,9223225.0,,,2025-10-30T11:31:41.660849
7,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2016,8935095.0,,,2025-10-30T11:31:41.660849
8,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2015,8505237.0,,,2025-10-30T11:31:41.660849
9,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2014,8059440.0,,,2025-10-30T11:31:41.660849


In [23]:
# Summary statistics
print("Summary Statistics:")
print("="*60)
print(combined_df.groupby('indicator_name').agg({
    'value': ['count', 'mean', 'min', 'max'],
    'country_id': 'nunique'
}).round(2))

Summary Statistics:
                                                   value                \
                                                   count          mean   
indicator_name                                                           
GDP (current US$)                                     98  3.585679e+11   
GDP per capita (current US$)                          98  1.317574e+04   
Life expectancy at birth, total (years)               98  6.903000e+01   
Literacy rate, adult total (% of people ages 15...    28  8.834000e+01   
Mortality rate, under-5 (per 1,000 live births)       98  3.720000e+01   
Population, total                                     98  6.482298e+07   

                                                                  \
                                                             min   
indicator_name                                                     
GDP (current US$)                                   2.713380e+10   
GDP per capita (current US$)             

## 6. Data Transformation with Python

Add calculated fields and clean the data for analysis.

In [24]:
# Create a copy for transformation
df_transformed = combined_df.copy()

# Add decade column
df_transformed['decade'] = (df_transformed['year'] // 10) * 10

# Add region classification (simplified)
region_mapping = {
    'EG': 'Middle East & North Africa',
    'SA': 'Middle East & North Africa',
    'AE': 'Middle East & North Africa',
    'JO': 'Middle East & North Africa',
    'NG': 'Sub-Saharan Africa',
    'ZA': 'Sub-Saharan Africa',
    'KE': 'Sub-Saharan Africa'
}
df_transformed['region'] = df_transformed['country_id'].map(region_mapping)

# Add category for indicators
category_mapping = {
    'SP.POP.TOTL': 'Demographics',
    'NY.GDP.MKTP.CD': 'Economy',
    'NY.GDP.PCAP.CD': 'Economy',
    'SP.DYN.LE00.IN': 'Health',
    'SE.ADT.LITR.ZS': 'Education',
    'SH.DYN.MORT': 'Health',
    'EN.ATM.CO2E.PC': 'Environment'
}
df_transformed['category'] = df_transformed['indicator_id'].map(category_mapping)

# Remove any null values
df_transformed = df_transformed.dropna(subset=['value'])

# Add unique record ID
df_transformed['record_id'] = df_transformed.apply(
    lambda row: f"{row['country_id']}_{row['indicator_id']}_{row['year']}", 
    axis=1
)

print(f"✓ Transformation complete")
print(f"  Original records: {len(combined_df)}")
print(f"  Transformed records: {len(df_transformed)}")
print(f"  New columns added: decade, region, category, record_id")

df_transformed.head()

✓ Transformation complete
  Original records: 518
  Transformed records: 518
  New columns added: decade, region, category, record_id


,indicator_id,indicator_name,country_id,country_name,year,value,unit,obs_status,fetched_at,decade,region,category,record_id
0,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2023,10483751.0,,,2025-10-30T11:31:41.660849,2020,Middle East & North Africa,Demographics,AE_SP.POP.TOTL_2023
1,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2022,10074977.0,,,2025-10-30T11:31:41.660849,2020,Middle East & North Africa,Demographics,AE_SP.POP.TOTL_2022
2,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2021,9575152.0,,,2025-10-30T11:31:41.660849,2020,Middle East & North Africa,Demographics,AE_SP.POP.TOTL_2021
3,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2020,9401038.0,,,2025-10-30T11:31:41.660849,2020,Middle East & North Africa,Demographics,AE_SP.POP.TOTL_2020
4,SP.POP.TOTL,"Population, total",AE,United Arab Emirates,2019,9445785.0,,,2025-10-30T11:31:41.660849,2010,Middle East & North Africa,Demographics,AE_SP.POP.TOTL_2019


## 7. Save Data to CSV for Snowflake Loading

In [25]:
# Save the transformed data
output_file = DATA_DIR / "worldbank_indicators.csv"
df_transformed.to_csv(output_file, index=False)

print(f"✓ Data saved to: {output_file}")
print(f"  File size: {output_file.stat().st_size / 1024:.2f} KB")
print(f"  Records: {len(df_transformed)}")
print(f"  Columns: {len(df_transformed.columns)}")

✓ Data saved to: ..\data\worldbank_indicators.csv
  File size: 85.34 KB
  Records: 518
  Columns: 13

  File size: 85.34 KB
  Records: 518
  Columns: 13


## 8. Configure Snowflake Connection

Set up connection parameters for Snowflake.

In [26]:
# Snowflake configuration (using securely entered password)
SNOWFLAKE_CONFIG = {
    'account': os.getenv('SNOWFLAKE_ACCOUNT', 'YOUR_ACCOUNT'),
    'user': os.getenv('SNOWFLAKE_USER', 'YOUR_USERNAME'),
    'password': SNOWFLAKE_PASSWORD,  # Using securely entered password from above
    'role': os.getenv('SNOWFLAKE_ROLE', 'ACCOUNTADMIN'),
    'warehouse': os.getenv('SNOWFLAKE_WAREHOUSE', 'ANALYTICS_WH'),
    'database': os.getenv('SNOWFLAKE_DATABASE', 'ANALYTICS_DB'),
    'schema': os.getenv('SNOWFLAKE_SCHEMA', 'RAW')
}

def get_snowflake_connection():
    """Create and return a Snowflake connection."""
    try:
        conn = snowflake.connector.connect(
            account=SNOWFLAKE_CONFIG['account'],
            user=SNOWFLAKE_CONFIG['user'],
            password=SNOWFLAKE_CONFIG['password'],
            role=SNOWFLAKE_CONFIG['role'],
            warehouse=SNOWFLAKE_CONFIG['warehouse'],
            database=SNOWFLAKE_CONFIG['database'],
            schema=SNOWFLAKE_CONFIG['schema']
        )
        print(f"✓ Connected to Snowflake")
        print(f"  Account: {SNOWFLAKE_CONFIG['account']}")
        print(f"  Database: {SNOWFLAKE_CONFIG['database']}")
        print(f"  Schema: {SNOWFLAKE_CONFIG['schema']}")
        return conn
    except Exception as e:
        print(f"✗ Failed to connect to Snowflake: {e}")
        return None

# Test connection (comment out if credentials not set yet)
# conn = get_snowflake_connection()
# if conn:
#     conn.close()

print("✓ Snowflake configuration ready")

✓ Snowflake configuration ready


## 9. Create Snowflake Objects

Provision the necessary warehouse, database, schemas, and tables.

In [27]:
def provision_snowflake_objects(conn):
    """Create Snowflake objects for the World Bank pipeline."""
    cursor = conn.cursor()
    
    ddl_statements = [
        # Create warehouse
        """
        CREATE WAREHOUSE IF NOT EXISTS ANALYTICS_WH
            WITH WAREHOUSE_SIZE = 'XSMALL'
            AUTO_SUSPEND = 60
            AUTO_RESUME = TRUE
            INITIALLY_SUSPENDED = TRUE
        """,
        # Create database
        "CREATE DATABASE IF NOT EXISTS ANALYTICS_DB",
        "USE DATABASE ANALYTICS_DB",
        # Create schemas
        "CREATE SCHEMA IF NOT EXISTS RAW",
        "CREATE SCHEMA IF NOT EXISTS RAW_STAGING",
        "CREATE SCHEMA IF NOT EXISTS RAW_MARTS",
        "USE SCHEMA RAW",
        # Create table for World Bank data
        """
        CREATE TABLE IF NOT EXISTS WORLDBANK_INDICATORS (
            record_id VARCHAR(200) PRIMARY KEY,
            indicator_id VARCHAR(100),
            indicator_name VARCHAR(500),
            country_id VARCHAR(10),
            country_name VARCHAR(200),
            year NUMBER,
            value FLOAT,
            unit VARCHAR(100),
            obs_status VARCHAR(50),
            decade NUMBER,
            region VARCHAR(100),
            category VARCHAR(100),
            fetched_at TIMESTAMP_NTZ
        )
        """
    ]
    
    for i, stmt in enumerate(ddl_statements, 1):
        try:
            cursor.execute(stmt)
            first_line = stmt.strip().split('\\n')[0][:50]
            print(f"  ✓ Statement {i}: {first_line}...")
        except Exception as e:
            print(f"  ✗ Statement {i} failed: {e}")
            raise
    
    cursor.close()
    print("\\n✓ Snowflake objects provisioned successfully")

# Uncomment to run:
# conn = get_snowflake_connection()
# if conn:
#     provision_snowflake_objects(conn)
#     conn.close()

print("✓ Provisioning function ready")

✓ Provisioning function ready


## 10. Load Data to Snowflake

Upload the CSV data to Snowflake table.

In [28]:
def load_data_to_snowflake(conn, df: pd.DataFrame, table_name: str):
    """Load DataFrame to Snowflake table using CSV and COPY INTO."""
    import io
    
    try:
        cursor = conn.cursor()
        
        # Create CSV in memory
        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=False, header=False)
        csv_buffer.seek(0)
        
        # Clear existing data
        cursor.execute(f"TRUNCATE TABLE IF EXISTS {table_name}")
        print(f"✓ Cleared existing data from {table_name}")
        
        # Prepare column list for INSERT
        columns = ', '.join(df.columns)
        placeholders = ', '.join(['%s'] * len(df.columns))
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        
        # Load data in batches
        batch_size = 1000
        rows_loaded = 0
        batch_data = []
        
        for _, row in df.iterrows():
            batch_data.append(tuple(row))
            
            if len(batch_data) >= batch_size:
                cursor.executemany(insert_query, batch_data)
                rows_loaded += len(batch_data)
                print(f"  Loaded {rows_loaded} rows...", end='\r')
                batch_data = []
        
        # Load remaining rows
        if batch_data:
            cursor.executemany(insert_query, batch_data)
            rows_loaded += len(batch_data)
        
        cursor.close()
        print(f"\n✓ Successfully loaded {rows_loaded} rows to {table_name}")
        return rows_loaded
            
    except Exception as e:
        print(f"✗ Error loading data: {e}")
        import traceback
        traceback.print_exc()
        return 0

# Uncomment to run:
# conn = get_snowflake_connection()
# if conn:
#     rows_loaded = load_data_to_snowflake(conn, df_transformed, 'WORLDBANK_INDICATORS')
#     conn.close()

print("✓ Loading function ready")

✓ Loading function ready


## 11. Next Steps: dbt & Dagster Integration

Now that we have the data ready, we'll create:

1. **dbt models** for transformations (staging, dimensions, facts)
2. **Dagster assets** for orchestration
3. **Complete pipeline** that runs automatically

The following cells show the structure of these files. They will be created as separate files in the project.

## 12. Test Snowflake Connection

Make sure you've entered your password in **Cell 2** before running this cell.

In [29]:
# Test Snowflake connection
print("Testing Snowflake connection...")
print("="*60)

try:
    conn = get_snowflake_connection()
    if conn:
        cursor = conn.cursor()
        cursor.execute("SELECT CURRENT_VERSION()")
        version = cursor.fetchone()[0]
        print(f"✓ Successfully connected to Snowflake!")
        print(f"  Snowflake Version: {version}")
        print(f"  Account: {SNOWFLAKE_CONFIG['account']}")
        print(f"  Database: {SNOWFLAKE_CONFIG['database']}")
        print(f"  Schema: {SNOWFLAKE_CONFIG['schema']}")
        cursor.close()
        conn.close()
        print("\n✓ Connection test successful!")
    else:
        print("✗ Failed to establish connection")
except Exception as e:
    print(f"✗ Connection failed: {e}")
    print("\nPlease check your .env file and update with correct credentials:")

Testing Snowflake connection...


✓ Connected to Snowflake
  Account: POKMPXO-ICB41863
  Database: ANALYTICS_DB
  Schema: RAW
✓ Successfully connected to Snowflake!
  Snowflake Version: 9.34.0
  Account: POKMPXO-ICB41863
  Database: ANALYTICS_DB
  Schema: RAW
✓ Successfully connected to Snowflake!
  Snowflake Version: 9.34.0
  Account: POKMPXO-ICB41863
  Database: ANALYTICS_DB
  Schema: RAW

✓ Connection test successful!

✓ Connection test successful!


## 13. Provision Snowflake Objects

Create the warehouse, database, schemas, and tables in Snowflake.

In [30]:
# Provision Snowflake objects
print("Provisioning Snowflake objects...")
print("="*60)

try:
    conn = get_snowflake_connection()
    if conn:
        provision_snowflake_objects(conn)
        conn.close()
        print("\n" + "="*60)
        print("✓ All Snowflake objects created successfully!")
        print("="*60)
        print("\nCreated objects:")
        print("  - Warehouse: ANALYTICS_WH")
        print("  - Database: ANALYTICS_DB")
        print("  - Schemas: RAW, RAW_STAGING, RAW_MARTS")
        print("  - Table: ANALYTICS_DB.RAW.WORLDBANK_INDICATORS")
except Exception as e:
    print(f"\n✗ Provisioning failed: {e}")
    print("Please check your Snowflake credentials and permissions.")

Provisioning Snowflake objects...
✓ Connected to Snowflake
  Account: POKMPXO-ICB41863
  Database: ANALYTICS_DB
  Schema: RAW
✓ Connected to Snowflake
  Account: POKMPXO-ICB41863
  Database: ANALYTICS_DB
  Schema: RAW
  ✓ Statement 1: CREATE WAREHOUSE IF NOT EXISTS ANALYTICS_WH
      ...
  ✓ Statement 1: CREATE WAREHOUSE IF NOT EXISTS ANALYTICS_WH
      ...
  ✓ Statement 2: CREATE DATABASE IF NOT EXISTS ANALYTICS_DB...
  ✓ Statement 2: CREATE DATABASE IF NOT EXISTS ANALYTICS_DB...
  ✓ Statement 3: USE DATABASE ANALYTICS_DB...
  ✓ Statement 3: USE DATABASE ANALYTICS_DB...
  ✓ Statement 4: CREATE SCHEMA IF NOT EXISTS RAW...
  ✓ Statement 4: CREATE SCHEMA IF NOT EXISTS RAW...
  ✓ Statement 5: CREATE SCHEMA IF NOT EXISTS RAW_STAGING...
  ✓ Statement 5: CREATE SCHEMA IF NOT EXISTS RAW_STAGING...
  ✓ Statement 6: CREATE SCHEMA IF NOT EXISTS RAW_MARTS...
  ✓ Statement 6: CREATE SCHEMA IF NOT EXISTS RAW_MARTS...
  ✓ Statement 7: USE SCHEMA RAW...
  ✓ Statement 7: USE SCHEMA RAW...
  ✓ Statemen

## 14. Load Data to Snowflake

Upload the transformed data to the Snowflake table.

In [31]:
# Load the transformed data to Snowflake
print("Loading data to Snowflake...")
print("="*60)

try:
    conn = get_snowflake_connection()
    if conn:
        rows_loaded = load_data_to_snowflake(conn, df_transformed, 'WORLDBANK_INDICATORS')
        conn.close()
        
        if rows_loaded > 0:
            print("\n" + "="*60)
            print("✅ Data loading completed successfully!")
            print(f"📊 Loaded {rows_loaded} records to WORLDBANK_INDICATORS")
            print("="*60)
        else:
            print("❌ Data loading failed. Check the error messages above.")
    else:
        print("❌ Could not establish connection to Snowflake")
except Exception as e:
    print(f"❌ Error during data loading: {e}")
    print("Make sure you've entered your password in Cell 2 and that the Snowflake objects were provisioned successfully.")

Loading data to Snowflake...
✓ Connected to Snowflake
  Account: POKMPXO-ICB41863
  Database: ANALYTICS_DB
  Schema: RAW
✓ Connected to Snowflake
  Account: POKMPXO-ICB41863
  Database: ANALYTICS_DB
  Schema: RAW
✓ Cleared existing data from WORLDBANK_INDICATORS
✓ Cleared existing data from WORLDBANK_INDICATORS

✓ Successfully loaded 518 rows to WORLDBANK_INDICATORS

✓ Successfully loaded 518 rows to WORLDBANK_INDICATORS

✅ Data loading completed successfully!
📊 Loaded 518 records to WORLDBANK_INDICATORS

✅ Data loading completed successfully!
📊 Loaded 518 records to WORLDBANK_INDICATORS


## 15. Verify Data in Snowflake

Query the table to confirm data was loaded correctly.

In [32]:
# Verify data was loaded
print("Verifying data in Snowflake...")
print("="*60)

try:
    conn = get_snowflake_connection()
    cursor = conn.cursor()
    
    # Count rows
    cursor.execute("SELECT COUNT(*) FROM WORLDBANK_INDICATORS")
    row_count = cursor.fetchone()[0]
    print(f"\n✅ Total records in WORLDBANK_INDICATORS: {row_count}")
    
    # Show sample data
    print("\n📋 Sample data (first 5 rows):")
    cursor.execute("SELECT record_id, country_name, indicator_name, year, value FROM WORLDBANK_INDICATORS LIMIT 5")
    for row in cursor:
        print(row)
    
    cursor.close()
    conn.close()
    
except Exception as e:
    print(f"❌ Error verifying data: {e}")

Verifying data in Snowflake...
✓ Connected to Snowflake
  Account: POKMPXO-ICB41863
  Database: ANALYTICS_DB
  Schema: RAW
✓ Connected to Snowflake
  Account: POKMPXO-ICB41863
  Database: ANALYTICS_DB
  Schema: RAW

✅ Total records in WORLDBANK_INDICATORS: 518

📋 Sample data (first 5 rows):

✅ Total records in WORLDBANK_INDICATORS: 518

📋 Sample data (first 5 rows):
('AE_SP.POP.TOTL_2023', 'United Arab Emirates', 'Population, total', 2023, 10483751.0)
('AE_SP.POP.TOTL_2022', 'United Arab Emirates', 'Population, total', 2022, 10074977.0)
('AE_SP.POP.TOTL_2021', 'United Arab Emirates', 'Population, total', 2021, 9575152.0)
('AE_SP.POP.TOTL_2020', 'United Arab Emirates', 'Population, total', 2020, 9401038.0)
('AE_SP.POP.TOTL_2019', 'United Arab Emirates', 'Population, total', 2019, 9445785.0)
('AE_SP.POP.TOTL_2023', 'United Arab Emirates', 'Population, total', 2023, 10483751.0)
('AE_SP.POP.TOTL_2022', 'United Arab Emirates', 'Population, total', 2022, 10074977.0)
('AE_SP.POP.TOTL_2021', '

## 16. Run dbt Transformations

Now that the raw data is loaded, run dbt to create the staging, dimension, and fact tables.

**To run dbt transformations:**

1. Open a terminal (PowerShell)
2. Navigate to the dbt directory:
   ```powershell
   cd dbt
   ```

3. Run the dbt models:
   ```powershell
   dbt run
   ```

4. Run the dbt tests:
   ```powershell
   dbt test
   ```

5. View the documentation (optional):
   ```powershell
   dbt docs generate
   dbt docs serve
   ```

This will create:
- **Staging**: `STG_WORLDBANK` view in `RAW_DATA` schema
- **Dimensions**: `DIM_COUNTRY` and `DIM_INDICATOR` tables in `RAW_DATA` schema
- **Fact Table**: `FACT_WORLDBANK` table in `RAW_MARTS` schema

## 17. Next Steps & Automation

### ✅ Completed in this Notebook:
1. ✓ Fetched data from World Bank API
2. ✓ Transformed and cleaned the data
3. ✓ Connected to Snowflake
4. ✓ Provisioned database objects
5. ✓ Loaded data to Snowflake

### 📋 Next Steps:

**1. Run dbt transformations** (see cell above)
   - Creates staging, dimension, and fact tables
   - Validates data with tests

**2. Set up Dagster for Automation** (optional)
   ```powershell
   cd orchestration/dagster
   dagster dev
   ```
   - Open http://localhost:3000 in your browser
   - Materialize all assets to run the full pipeline
   - Schedule automated runs

**3. Create Visualizations**
   - Use Power BI to connect to Snowflake
   - Or create Python visualizations in a new notebook
   - Query the `FACT_WORLDBANK` table for analytics

**4. Explore the Data**
   - Connect to Snowflake using SnowSQL or web UI
   - Query dimension and fact tables
   - Create custom analytics queries

### 📚 Documentation:
- **PIPELINE_README.md** - Complete pipeline architecture
- **WORLDBANK_QUICKSTART.md** - World Bank API reference
- **README.md** - Project quick start guide

### Summary

✅ **Completed:**
- Fetched data from World Bank API
- Transformed and enriched the data with Python
- Prepared data for Snowflake loading
- Created functions for Snowflake provisioning and data loading

📝 **Files to Create Next:**
1. `dbt/models/staging/stg_worldbank.sql` - Staging model
2. `dbt/models/dimensions/dim_country.sql` - Country dimension
3. `dbt/models/dimensions/dim_indicator.sql` - Indicator dimension
4. `dbt/models/marts/fact_worldbank.sql` - Fact table
5. `orchestration/dagster/worldbank_pipeline.py` - Dagster assets